# Construction of the Conclave Network
**Blas Kolic, 2025** <br>
*Adapted from the [Conclave analysis Jupyter notebook (2025)](https://github.com/leonardorizzo/Conclave2025/blob/main/conclave_analysis.ipynb) by Giuseppe Soda, Alessandro Iorio, and Leonardo Rizzo*

A group of Bocconi scholars (*Giuseppe Soda, Alessandro Iorio, and Leonardo Rizzo*) analyzed the social network of the Conclave after Pope Francis died. In this network, a node is a person in the College of Cardinals, i.e., a potential candidate for being the new pope, and edges are multilayered: edges in the first layer represent the *episcopal consecration* between cardinals (i.e., which cardinals where ordained by whom), while an edge in the second layer represents the number of *co-memberships* between cardinals (i.e., on how many institutions the cardinals work together). The network we will analyze combines this layeres into a singled-layered edge by summing the weight of each edge.

This notebook walks you through on how to construct the final network based on the data shared on the [GitHub repository](https://github.com/leonardorizzo/Conclave2025/) of the Bocconi scholars. For more details, [read their paper](linkinghub.elsevier.com/retrieve/pii/S0378873325000449).

In [1]:
import networkx as nx
import pandas as pd

In [ ]:
# 0. Set relevant paths
fn_consecration = '../data/conclave/raw/adjacency_matrix_consecrator.csv'
fn_memberships = '../data/conclave/raw/formal.csv'

# 1. Create Consecration Network (unweighted adjacency provided)
A_consecration = pd.read_csv(fn_consecration, index_col=0).astype('int')
G_consecration = nx.from_pandas_adjacency(A_consecration, create_using=nx.Graph())

# 2. Create co-membership network (Person to Membership edgelist provided)
df_c2m = pd.read_csv(fn_memberships)
## First, build bipartite graph from edgelist
G_c2m = nx.from_pandas_edgelist(df_c2m, source="Person", target="Membership")
## Define the two node sets explicitly
candidates, memberships  = set( df_c2m["Person"] ), set( df_c2m["Membership"] )
## Project into candidate co-membership network
G_comemberships = nx.bipartite.weighted_projected_graph(G_c2m, candidates)

# 3. Add the networks together (add edges and weights)
G_combined = nx.Graph() # Initialize an empty graph
## First, add edges from G_consecration (weight=1)
for (u, v) in G_consecration.edges():
    G_combined.add_edge(u, v, weight=1)
## Then, add weighted edges from G_comemberships (adds or sums)
for (u, v, w) in G_comemberships.edges(data='weight'):    
    # If edge exists already, add weight
    if G_combined.has_edge(u, v):
        G_combined[u][v]["weight"] += w
    # Otherwise, create the (weighted) edge
    else:
        G_combined.add_edge(u, v, weight=w)

# 4. Save network as an edgelist (Optional)
## Transform network back to a pandas edgelist
edgelist_combined = nx.to_pandas_edgelist(G_combined)
## Save it
edgelist_combined.to_csv('../data/conclave/edgelist_conclave_final.csv', index=False)
print("Edgelist saved successfully!")
edgelist_combined.head() # show first few rows

Edgelist saved successfully!


,source,target,weight
0,Aguiar Alves,Clemente,1
1,Aguiar Alves,Gambetti,1
2,Aguiar Alves,Bo,1
3,Aguiar Alves,Langlois,1
4,Aguiar Alves,Semeraro,1
